<table style="width:100%; border:0" cellspacing="0" cellpadding="0" border="0" align="left|left|right">
  <tr style="border:0">
  <td style="border: 0; font-size:60px" rowspan="3"><b>Universität Bielefeld</b></td>
    <td style="border: 0">CITEC / Faculty of Technology</td>
  </tr>
  <tr style="border:0">
    <td style="border: 0">Multimodal Behavior Processing Group</td>
  </tr>
</table>

# Model Interpretability
In this notebook you will use some model interpretability methods for exploring the predictions of the Adult dataset with [Lime](https://github.com/marcotcr/lime) and the [CelebA](https://mmlab.ie.cuhk.edu.hk/projects/CelebA.html) image dataset using the [innvestigate](https://github.com/albermax/innvestigate) library.

__NOTE__: Before getting started make sure you have followed the instructions in the `Model_interpretability_README.md` and validated your conda installation with `Model_interpretability_Lib_validation.ipynb`


## LIME
[Lime](https://arxiv.org/abs/1602.04938) stands for local interpretable model-agnostic explanations, meaning that it can be applied to any machine learning model and explains the model's decision for every local sample. By modifying the features of a single sample LIME observes what will happen to the predicted outcome. It then takes the modified dataset to train an interpretable model wich is a good approximation of the local predictions but not global approximation of the model.


You will use the Adult dataset with which you are familiar now to explore the explanations of the predictions before and after using the debiasing techniques from the last notebook. 

In [ ]:
from aif360.datasets import AdultDataset 
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from IPython.display import Markdown, display
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
# protected features
protected = 'sex'
# privileged class 
privileged_classes = [['Male']]
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]
features_to_keep=['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']

# load the dataset 
ad = AdultDataset(protected_attribute_names=[protected],
    privileged_classes=privileged_classes, categorical_features=[],
    features_to_keep=features_to_keep)
 

In [ ]:
ad_df = ad.convert_to_dataframe()[0]
ad_df.sample(10)

In [ ]:
# split the data into train , valid and test set 
ad_train, ad_test = ad.split([0.7], shuffle = False, seed=0)
ad_valid, ad_test = ad_test.split([0.5], shuffle=False, seed=0)

#### Train the Block Box Classifier

In [ ]:
# load features (For this example we don't scale the data)
x_train = ad_train.features
y_train = ad_train.labels.ravel()

# train the model
clf = RandomForestClassifier()   # a black-box classifier
clf.fit(x_train, y_train, sample_weight=ad_train.instance_weights)
print(f'Accuracy = {clf.score(x_train, y_train)}')

#### Use Lime to Generate Explanations

In [ ]:
from lime.lime_tabular import LimeTabularExplainer
from lime import submodular_pick

### Task 3.1.1: Use the Tabular Lime Explainer within the classifier function to explain a local sample. Describe the visualisations of the explanations and interpret them. 

For the explanation part we only need the classifier function wich takes the classfifier, the train set and the sample number to be explained by Lime. 
You will need to use the Lime Tabular Explainer which explains classifiers that use tabular data like the Adult dataset. You can familiarize yourself with the modules of Lime [here](https://lime-ml.readthedocs.io/en/latest/index.html). 

*Hint: The visualiazed data is now scaled but you can still explain their relation (e.g. for age and sex data) or have a look at the original data of that sample. 


In [ ]:
def lime_explainer(classifier, x_train, sample_number):
    
    """
    Apply the LimeTabularExplainer
    
    """
    
    #####################################################
    #                   Your code                       #
    #####################################################

### Task 3.1.2: Use the Submodular Pick module within the classifier function to approximate a global explanation. Describe the visualisations of the explanations and interpret them. 

To get a global view of which features 
Submodular Pick combines local explanations by averaging the contribution of each 

You will find an example how to use Submodular pick [here](https://github.com/marcotcr/lime/blob/master/doc/notebooks/Submodular%20Pick%20examples.ipynb).

*Hint: Have a look at the parameters and decide which sample size is appropriate for the trade of global approximation and calculation time. 

In [ ]:
def submodular_explainer(classifier, x_train, sample_number):
    
   
    """
    Apply the SubmodularPick module
    
    """
    
    #####################################################
    #                   Your code                       #
    #####################################################

#### Get Local Explanations About the Prediction

In [ ]:
# chose a sample from the dataset
sample_number = 100
lime_explainer(clf, x_train, sample_number)

#### Get Global View of Model with Submodular Pick

In [ ]:
submodular_explainer(clf, x_train, sample_number)

#### Discuss your Results

Your solution for Task 3.1.1:


Your solution for Task 3.1.2:


### Task 3.1.3: Use a preprocessing bias mitigation technique which was introduced in the last notebook and apply both the local and global explanation. Have a look at a sample which belongs to an unprivileged group and compare the explained prediction before and after debiasing. 

In [ ]:
# after debiasing

from aif360.algorithms.preprocessing.reweighing import Reweighing
from aif360.algorithms.preprocessing import DisparateImpactRemover

# check the documentation and examples of how to use the techniques 
def preprocessing(train, valid, test, unprivileged_groups, privileged_groups,method=None, repair_level=1,
                  k=5, Ax=0.01, Ay=1.0, Az=50.0, print_interval=250, verbose=1, seed=0):
    
    if method=="Reweighing":
        RW = Reweighing(unprivileged_groups=unprivileged_groups,privileged_groups=privileged_groups)
        RW.fit(train)
        return RW.transform(train), valid, test
    
    elif method=="DisparateImpactRemover":
        DI = DisparateImpactRemover(repair_level=repair_level)
        return DI.fit_transform(train), DI.fit_transform(valid), DI.fit_transform(test)
    
    else:
        print("The preprocessing methods is not implemented")

#### Retrain Model with Dibiasing

In [ ]:
ad_train_re, ad_valid_re, ad_test_re = preprocessing(ad_train, ad_valid, ad_test, unprivileged_groups,privileged_groups, method="Reweighing")

# load features (For this example we don't scale the data)
x_train = ad_train.features
y_train = ad_train.labels.ravel()

# train the model
clf = RandomForestClassifier()   # a black-box classifier
clf.fit(x_train, y_train, sample_weight=ad_train.instance_weights)
print(f'Accuracy = {clf.score(x_train, y_train)}')

#### Get Local Explanations About the Debiased Prediction

In [ ]:
# chose a sample from the dataset
sample_number = 100
lime_explainer(clf, x_train, sample_number)

#### Get Global View of Model with Submodular Pick

In [ ]:
submodular_explainer(clf, x_train, sample_number)

#### Discuss Your Results

Your solution for Task 3.1.3:

## Image Based Explanations with Innvestigate

[Innvestigate](https://github.com/albermax/innvestigate) provides several methods to understand neural networks such as DevConvnet, Deeptaylor and LRP. 
For this part we will focus on LRP only. 
LPR stands for Layerwise Relevance Propagation and with LRP the classification decision of a model is backpropagated in order to get information about features which had significant impact on the prediction. For image classification, you will get a heatmap with relevance values which are calculated for every backpropagated layer.
LRP is a model specific method and needs different LRP-calculations. 
There is good explanation [here](https://towardsdatascience.com/indepth-layer-wise-relevance-propagation-340f95deb1ea)

For this assignment we well create a grid of visualzations from __three__ XAI methods to help explain why a certain prediction was made using available methods from the [iNNvestigate library](https://github.com/albermax/innvestigate). 

We will analyze an already trained model for detecting Arched Eyebrows from face images.  The model was trained using a subset of the [CelebA dataset](https://mmlab.ie.cuhk.edu.hk/projects/CelebA.html).

In [ ]:
# import necesssary libaries
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt

from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator

import innvestigate
import innvestigate.utils as iutils
import innvestigate.applications.imagenet
import innvestigate.utils.visualizations as ivis

### Load the Pretrained Model

First we need to load our pretrained model.  This model is trained to detect arched eyebrows in images of faces.  The model was trained using the CelebA dataset.

NOTE: If you have trouble loading model try running: `pip install h5py==2.10.0 --force-reinstall` at your terminal

In [ ]:
import keras
from keras.applications.mobilenet_v2 import MobileNetV2

# adding build model function to fix a bug: https://github.com/albermax/innvestigate/issues/177
def build_model(num_features):
  base = MobileNetV2(input_shape=(224, 224, 3),
                     weights=None,
                     include_top=False,
                     pooling='avg')  # GlobalAveragePooling 2D
  
  # model top
  x = base.output
  x = keras.layers.Dense(1536, activation='relu')(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Dropout(0.3)(x)
  top = keras.layers.Dense(num_features, activation='softmax')(x)
  
  return keras.models.Model(inputs=base.input, outputs=top)

In [ ]:
model = build_model(2)
model = load_model('../data/models/weights-FC2-best_model.hdf5')
# model.summary()  # uncomment if you're interested in details about the model

### Load Example Images

Next, we will load 10 example images for analysis extracted from the validation partition of the CelebA dataset.

In [ ]:
classes = ['Arched_Eyebrows', 'no_Arched_Eyebrows']

# rescaling data between 0 and 1
examples_datagen = ImageDataGenerator(rescale=1./255)

example_generator = examples_datagen.flow_from_directory(
    directory = '../data/celeba_examples',
    target_size = (224, 224),
    class_mode = 'categorical',
    classes = classes,
    batch_size = 1,
    shuffle = False
)

In [ ]:
### Bonus Task: Try loading your own images, i.e. pictures of yourself or other images that interest you

#####################################################
#                   Your code                       #
#####################################################

### Task 3.2.1
To get started with iNNvestigate, the final softmax layer of the model must be removed so that innvestigate can work with the model logits.
see the [innvestigate.utils module](https://innvestigate.readthedocs.io/en/latest/modules/utils.html#module-innvestigate.utils) for a helper function to remove the final softmax activation layer

In [ ]:
# Remove the softmax activation from the model
#####################################################
#                   Your code                       #
#####################################################

### Task 3.2.2
Next, to setup our analyis, we need to define three explainability methods to use, and corresponding parameters and post-processing methods.   For this task, fill in the lists below with 3 XAI methods of your choice, and their corresponding parameters and post-processing methods. You can use the following resources to help. 

- [iNNvestigate Analysis Tutorial Notebook](https://github.com/albermax/innvestigate/blob/master/examples/notebooks/imagenet_compare_methods.ipynb)
- [iNNvestigate API Reference](https://innvestigate.readthedocs.io/en/latest/index.html)

In [ ]:
#####################################################
#                   Your code                       #
#####################################################
# implement a list of 3 or more innvestigate explainability methods to use
methods = []

# a list dictionaries for keyword parameters for the corresponding methods
method_params = []

# a list dictionaries for keyword parameters for the corresponding methods
method_preprocessing = []

### Task 3.2.3
Now that you have your methods and parameters defined, we will analyze and visualize a set of images using your methods. Fill in the code sections below to perform your analysis and plot the results.

Use the above resources for help analysis via iNNvestigate, addtionally review the Matplotlib [imshow](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html) and [subplots](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.subplots.html) documentation to plot the analysis results.  

In [ ]:
n_images = len(example_generator)
n_images = 2  # uncomment to try with fewer images for faster results
    
# For each image loop through all methods to create visualization and save to a list
fig, axs = plt.subplots(n_images, len(methods) + 1, figsize=(16, 4.5 * n_images))

for y in range(n_images):
    image, label = example_generator[y]
    
    # get prediction
    prob = model.predict_on_batch(image)
    y_hat_cls = classes[prob.argmax()]
    y_cls = classes[label.argmax()]
    
    #####################################################
    #                   Your code                       #
    #####################################################
    # plot the raw image first and set the label to the 
    # predicted and ground truth classes.  

    
    # clean up visualization
    axs[y, 0].set_xticks([])
    axs[y, 0].set_yticks([])
    
    for x, (method, params, preprocess) in enumerate(zip(methods, method_params, method_preprocessing), start=1):
        
        #####################################################
        #                   Your code                       #
        #####################################################
        # create an analyzer for this method
        
        # analyze image 
        
        # preprocess image for visualization


        #####################################################
        #                   Your code                       #
        #####################################################
        # plot image and set a title with the method name
        
        
        # clean up visualization
        axs[y, x].set_xticks([])
        axs[y, x].set_yticks([])
plt.show()

### Task 3.2.4
Describe the three methods you selected and why you choose those particular methods.  Finally, describe your findings and interesting insights from implementing the methods for XAI, including how you think these methods enable (or not) interpretable machine learning.  

Your findings here...